In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
df = spark.createDataFrame([(1, '2023', 'jan', 'a1', 'reddy'), (2, '2023', 'jan', 'a2', 'reddy'), (3, '2023', 'feb', 'a3', 'reddy'), (4, '2023', 'feb', 'a4', 'reddy'), (5, '2024', 'jan', 'a5', 'reddy'), (6, '2024', 'jan', 'a6', 'reddy'), (7, '2024', 'feb', 'a7', 'reddy'), (8, '2024', 'feb', 'a8', 'reddy')], schema = ['id', 'year', 'month', 'fn', 'ln'])

df.display()



id,year,month,fn,ln
1,2023,jan,a1,reddy
2,2023,jan,a2,reddy
3,2023,feb,a3,reddy
4,2023,feb,a4,reddy
5,2024,jan,a5,reddy
6,2024,jan,a6,reddy
7,2024,feb,a7,reddy
8,2024,feb,a8,reddy


In [0]:
df1 = spark.createDataFrame([(1, '2023', 'jan', 'a1', 'reddy'), (2, '2023', 'jan', 'a2', 'reddy'), (4, '2023', 'feb', 'a4', 'reddy'), (5, '2024', 'jan', 'a5', 'reddy'), (6, '2024', 'jan', 'a6', 'reddy'), (8, '2024', 'feb', 'a8', 'reddy')], schema = ['id', 'year', 'month', 'fn', 'ln'])

In [0]:
df.write.format('delta').partitionBy('year', 'month').mode('overwrite').option('overwriteSchema', True).save('dbfs:/Users/arjun37ca@gmail.com/asd1')

In [0]:
years = df1.select("year").distinct().rdd.map(lambda row: row.year).collect()
months = df1.select("month").distinct().rdd.map(lambda row: row.month).collect()
list_of_year_and_month = [f"year == '{year}' AND month == '{month}'" for year, month in product(years, months)]
parts = [f"year={year}/month={month}/" for year, month in product(years, months)]


In [0]:
delta_table = DeltaTable.forPath(spark, f'dbfs:/Users/arjun37ca@gmail.com/asd1/year=2023/month=jan')


In [0]:
delta_table.toDF().display()

id,year,month,fn,ln
1,2023,jan,a1,reddy
2,2023,jan,a2,reddy


In [0]:
from itertools import  product
from delta.tables import DeltaTable
# delta_table = DeltaTable.forPath(spark, 'dbfs:/Users/arjun37ca@gmail.com/asd1')

for x, y in zip(list_of_year_and_month, parts):
    # print(year_month)
    # print(f'dbfs:/Users/arjun37ca@gmail.com/asd1/{year_month}')
    delta_table = DeltaTable.forPath(spark, f'dbfs:/Users/arjun37ca@gmail.com/asd1/')
    rdf = df1.filter(x)
    delta_table.alias('a1').merge(rdf.alias('a2'), 'a1.id == a2.id').whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()
        

In [0]:
print(*list_of_year_and_month)

year == '2023' AND month == 'jan' year == '2023' AND month == 'feb' year == '2024' AND month == 'jan' year == '2024' AND month == 'feb'


In [0]:
print(*parts)

year = 2023/month = jan year = 2023/month = feb year = 2024/month = jan year = 2024/month = feb


In [0]:
%fs ls dbfs:/Users/arjun37ca@gmail.com/asd1/year=2023/month=feb/

path,name,size,modificationTime
dbfs:/Users/arjun37ca@gmail.com/asd1/year=2023/month=feb/part-00000-2d85b0fa-e299-4532-8dfd-a626675a1964.c000.snappy.parquet,part-00000-2d85b0fa-e299-4532-8dfd-a626675a1964.c000.snappy.parquet,1047,1740501798000
dbfs:/Users/arjun37ca@gmail.com/asd1/year=2023/month=feb/part-00000-bd659f9d-7b09-4a64-90f9-353d56df1a46.c000.snappy.parquet,part-00000-bd659f9d-7b09-4a64-90f9-353d56df1a46.c000.snappy.parquet,1047,1740484253000
dbfs:/Users/arjun37ca@gmail.com/asd1/year=2023/month=feb/part-00002-70ca0c20-0909-42b1-b6ca-6d0895274de2.c000.snappy.parquet,part-00002-70ca0c20-0909-42b1-b6ca-6d0895274de2.c000.snappy.parquet,1048,1740482742000
dbfs:/Users/arjun37ca@gmail.com/asd1/year=2023/month=feb/part-00002-e6de6a4f-f4c3-4afa-b1b6-74953b5f249f.c000.snappy.parquet,part-00002-e6de6a4f-f4c3-4afa-b1b6-74953b5f249f.c000.snappy.parquet,1048,1740501762000
dbfs:/Users/arjun37ca@gmail.com/asd1/year=2023/month=feb/part-00003-c37bf344-5cbd-4d7b-997e-23d89211e206.c000.snappy.parquet,part-00003-c37bf344-5cbd-4d7b-997e-23d89211e206.c000.snappy.parquet,1047,1740482742000
dbfs:/Users/arjun37ca@gmail.com/asd1/year=2023/month=feb/part-00003-c4644535-b20e-4025-8ff1-e5c9dfe0751b.c000.snappy.parquet,part-00003-c4644535-b20e-4025-8ff1-e5c9dfe0751b.c000.snappy.parquet,1047,1740501762000


In [0]:
%sql
select * from delta.`dbfs:/Users/arjun37ca@gmail.com/asd1/year=2023` 

id,year,month,fn,ln
1,2023,jan,a1,reddy
2,2023,jan,a2,reddy
3,2023,feb,a3,reddy
4,2023,feb,a4,reddy


In [0]:
%sql
--delta_table = DeltaTable.forPath(spark, 'dbfs:/Users/arjun37ca@gmail.com/asd1')
select * from delta.`dbfs:/Users/arjun37ca@gmail.com/asd1`

id,year,month,fn,ln
1,2023,jan,a1,reddy
2,2023,jan,a2,reddy
5,2024,jan,a5,reddy
6,2024,jan,a6,reddy
3,2023,feb,a3,reddy
7,2024,feb,a7,reddy
8,2024,feb,a8,reddy
4,2023,feb,a4,reddy


In [0]:

# Sample dimension tables
product_data = [
    (1, "Product A"),
    (2, "Product B"),
    (3, "Product C")
]
time_data = [
    (1, 2025, 2, 24),
    (2, 2025, 2, 25)
]
region_data = [
    (1, "North"),
    (2, "South")
]

# Create DataFrames for dimension tables
product_df = spark.createDataFrame(product_data, ["product_id", "product_name"])
time_df = spark.createDataFrame(time_data, ["time_id", "year", "month", "day"])
region_df = spark.createDataFrame(region_data, ["region_id", "region_name"])

# Sample fact table (sales data)
sales_data = [
    (1, 1, 1, 1, 100.0),
    (2, 2, 2, 2, 150.0),
    (3, 1, 1, 2, 200.0)
]

# Create DataFrame for the fact table (sales)
sales_df = spark.createDataFrame(sales_data, ["sale_id", "product_id", "time_id", "region_id", "amount"])


In [0]:
# Join the fact table with dimension tables to enrich the data
fact_df = sales_df \
    .join(product_df, "product_id", "inner") \
    .join(time_df, "time_id", "inner") \
    .join(region_df, "region_id", "inner")

fact_df.display()


region_id,time_id,product_id,sale_id,amount,product_name,year,month,day,region_name
1,1,1,1,100.0,Product A,2025,2,24,North
2,2,2,2,150.0,Product B,2025,2,25,South
2,1,1,3,200.0,Product A,2025,2,24,South


In [0]:
total_sales_kpi = fact_df.groupBy("product_name", "region_name", "year", "month") \
    .agg(
        sum(col("amount").cast(IntegerType())).alias("total_sales")
    )

total_sales_kpi.display()


product_name,region_name,year,month,total_sales
Product A,North,2025,2,100
Product B,South,2025,2,150
Product A,South,2025,2,200


In [0]:
avg_sale_value_kpi = fact_df.groupBy("product_name", "region_name", "year", "month") \
    .agg(
        (sum(col("amount").cast(IntegerType())) / count("sale_id")).alias("avg_sale_value")
    )

avg_sale_value_kpi.display()


product_name,region_name,year,month,avg_sale_value
Product A,North,2025,2,100.0
Product B,South,2025,2,150.0
Product A,South,2025,2,200.0


In [0]:
sales_count_kpi = fact_df.groupBy("product_name", "region_name", "year", "month") \
    .agg(
        count("sale_id").alias("sales_count")
    )

sales_count_kpi.display()


product_name,region_name,year,month,sales_count
Product A,North,2025,2,1
Product B,South,2025,2,1
Product A,South,2025,2,1


In [0]:
final_report = total_sales_kpi \
    .join(avg_sale_value_kpi, ["product_name", "region_name", "year", "month"]) \
    .join(sales_count_kpi, ["product_name", "region_name", "year", "month"])

final_report.display()


product_name,region_name,year,month,total_sales,avg_sale_value,sales_count
Product A,North,2025,2,100,100.0,1
Product B,South,2025,2,150,150.0,1
Product A,South,2025,2,200,200.0,1
